In [80]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from tkinter.font import Font
import psycopg2
from bs4 import BeautifulSoup
import bs4
import os
from tkinterdnd2 import TkinterDnD, DND_FILES
import re
import datetime

def parse_date_string(date_string):

    date_string = date_string.lower()  # Для упрощения сравнения

    if "сегодня" in date_string:
        return datetime.date.today()
    elif "вчера" in date_string:
        return datetime.date.today() - datetime.timedelta(days=1)
    else:
        # Пытаемся обработать формат "ДД месяц ГГГГ"
        match = re.search(r"был (\d{1,2})\xa0(.*?)\xa0(\d{4})", date_string)
        if match:
            day = int(match.group(1))
            month_name = match.group(2).strip()
            year = int(match.group(3))

            month_dict = {
                "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
                "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,
            }

            if month_name in month_dict:
                month = month_dict[month_name]
                try:
                    return datetime.date(year, month, day)
                except ValueError:
                    return None

        else:
            # Пытаемся обработать формат "ДД месяц"
            match = re.search(r"был (\d{1,2})\xa0(.*)", date_string)
            if match:
                day = int(match.group(1))
                month_name = match.group(2).strip()

                month_dict = {
                    "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
                    "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,
                }

                if month_name in month_dict:
                    month = month_dict[month_name]
                    year = datetime.date.today().year
                    try:
                        return datetime.date(year, month, day)
                    except ValueError:
                        return None

    return None  # Если не удалось распознать дату


class DnDFrame(ttk.LabelFrame):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.drop_target_register(DND_FILES)
        self.dnd_bind('<<Drop>>', self.handle_drop)
        
        self.file_listbox = tk.Listbox(self, height=5)
        self.file_listbox.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        btn_frame = ttk.Frame(self)
        btn_frame.pack(fill=tk.X, pady=5)
        
        self.clear_btn = ttk.Button(btn_frame, text="Очистить", command=self.clear_files)
        self.clear_btn.pack(side=tk.LEFT, padx=5)
        
        self.add_btn = ttk.Button(btn_frame, text="Добавить файлы", command=self.add_files)
        self.add_btn.pack(side=tk.RIGHT, padx=5)
    
    def handle_drop(self, event):
        files = self.tk.splitlist(event.data)
        for file in files:
            if file.lower().endswith('.html'):
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
            else:
                messagebox.showwarning("Неверный формат", f"Файл {file} не является HTML")
    
    def add_files(self):
        filetypes = (('HTML files', '*.html'), ('All files', '*.*'))
        files = filedialog.askopenfilenames(title='Выберите файлы', filetypes=filetypes)
        if files:
            for file in files:
                if file not in self.file_listbox.get(0, tk.END):
                    self.file_listbox.insert(tk.END, file)
    
    def clear_files(self):
        self.file_listbox.delete(0, tk.END)
    
    def get_files(self):
        return list(self.file_listbox.get(0, tk.END))

class Application(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()
        self.title("Парсер вакансий HH")
        self.geometry("600x600")  # Увеличили высоту окна
        self.configure(bg='#f0f0f0')
        
        self.DB_CONFIG = {
            'host': 'localhost',
            'port': '5433',
            'database': 'hh_test',
            'user': 'postgres',
            'password': '123'
        }
        
        self.current_comment = ""  # Текущий комментарий для сессии парсинга
        self.create_widgets()
    
    def create_widgets(self):
        # Стилизация
        bold_font = Font(family="Arial", size=10, weight="bold")
        normal_font = Font(family="Arial", size=10)
        
        # Главный фрейм
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Заголовок
        ttk.Label(
            main_frame,
            text="Парсер вакансий HeadHunter",
            font=Font(family="Arial", size=12, weight="bold")
        ).pack(pady=(0, 10))
        
        # Область для комментария
        comment_frame = ttk.LabelFrame(main_frame, text="Комментарий к парсингу", padding=10)
        comment_frame.pack(fill=tk.X, pady=5, padx=5)
        
        self.comment_text = tk.Text(comment_frame, height=4, wrap=tk.WORD)
        self.comment_text.pack(fill=tk.BOTH, expand=True)
        
        ttk.Button(comment_frame, text="Сохранить комментарий", 
                 command=self.save_comment).pack(pady=5)
        
        # Область для перетаскивания файлов
        drag_frame = DnDFrame(main_frame, text="Перетащите HTML-файлы сюда или нажмите кнопку", 
                             width=500, height=150)
        drag_frame.pack(fill=tk.BOTH, pady=10, padx=10, expand=True)
        self.drag_frame = drag_frame
        
        # Кнопка обработки
        self.process_btn = ttk.Button(
            main_frame,
            text="Обработать файлы",
            command=self.process_files,
            style='Accent.TButton'
        )
        self.process_btn.pack(pady=10)
        
        # Статус бар
        self.status_var = tk.StringVar()
        status_bar = ttk.Label(
            main_frame,
            textvariable=self.status_var,
            relief=tk.SUNKEN,
            anchor=tk.W
        )
        status_bar.pack(fill=tk.X, pady=(10, 0))
        
        # Стиль
        style = ttk.Style()
        style.configure('Accent.TButton', font=bold_font, foreground='black')
    
    def save_comment(self):
        """Сохраняет комментарий для текущей сессии парсинга"""
        self.current_comment = self.comment_text.get("1.0", tk.END).strip()
        messagebox.showinfo("Сохранено", "Комментарий сохранен и будет добавлен ко всем обрабатываемым файлам")
    def process_files(self):
        files = self.drag_frame.get_files()
        if not files:
            messagebox.showwarning("Ошибка", "Нет файлов для обработки")
            return

        total_records = 0
        processed_files = 0
        file_types = set()
        session_id = None

        self.process_btn.config(state=tk.DISABLED)
        self.status_var.set("Начата обработка файлов...")
        self.update()

        try:
            # Получаем следующий ID сессии из БД
            with psycopg2.connect(**self.DB_CONFIG) as conn:
                with conn.cursor() as cur:
                    cur.execute("SELECT COALESCE(MAX(session_id), 0) FROM info_res")
                    session_id = cur.fetchone()[0] + 1

            for file in files:
                try:
                    data, metadata = self.parser(file)
                    if not data:
                        continue  # Пропускаем файлы без данных
                    
                    # Загрузка в БД
                    success = self.load_db(data, metadata, session_id)
                    if success:
                        total_records += len(data)
                        processed_files += 1
                        file_types.add(metadata[4])  # Тип файла
                        self.status_var.set(f"Обработан {os.path.basename(file)} ({len(data)} записей)")
                        self.update()
                    else:
                        messagebox.showerror("Ошибка", f"Ошибка при сохранении данных из {file}")
                except Exception as e:
                    messagebox.showerror("Ошибка", f"Ошибка при обработке {file}: {str(e)}")

            self.status_var.set(f"Готово! Обработано {processed_files}/{len(files)} файлов, {total_records} записей")

            # Показываем окно с результатами
            self.show_results(session_id, file_types, processed_files, len(files), total_records)

        except Exception as e:
            messagebox.showerror("Критическая ошибка", f"Ошибка при обработке: {str(e)}")
        finally:
            self.process_btn.config(state=tk.NORMAL)
            self.current_comment = ""  # Сбрасываем комментарий

    def show_results(self, session_id, file_types, processed_count, total_files, total_records):
        """Показывает окно с результатами обработки"""
        result_window = tk.Toplevel(self)
        result_window.title("Результаты обработки")
        result_window.geometry("400x300")
        result_window.resizable(False, False)

        # Стилизация
        style = ttk.Style()
        style.configure("Result.TLabel", font=("Arial", 10))
        style.configure("ResultHeader.TLabel", font=("Arial", 10, "bold"))

        # Заголовок
        ttk.Label(
            result_window,
            text="Результаты обработки",
            font=("Arial", 12, "bold"),
            justify=tk.CENTER
        ).pack(pady=(10, 20))

        # Фрейм для информации
        info_frame = ttk.Frame(result_window)
        info_frame.pack(padx=20, pady=5, fill=tk.X)

        # Данные для отображения
        results = [
            ("Номер сессии:", str(session_id)),
            ("Тип файлов:", ", ".join(file_types) if file_types else "Не определен"),
            ("Обработано файлов:", f"{processed_count} из {total_files}"),
            ("Добавлено записей:", str(total_records)),
            ("Дата обработки:", datetime.datetime.now().strftime("%d.%m.%Y %H:%M"))
        ]

        # Отображаем информацию
        for i, (label, value) in enumerate(results):
            ttk.Label(info_frame, text=label, style="ResultHeader.TLabel").grid(row=i, column=0, sticky=tk.W, pady=2)
            ttk.Label(info_frame, text=value, style="Result.TLabel").grid(row=i, column=1, sticky=tk.W, pady=2)

        # Кнопка закрытия
        ttk.Button(
            result_window,
            text="Закрыть",
            command=result_window.destroy,
            style="Accent.TButton"
        ).pack(pady=20)

        # Центрируем окно
        self.center_window(result_window)

    def center_window(self, window):
        """Центрирует окно на экране"""
        window.update_idletasks()
        width = window.winfo_width()
        height = window.winfo_height()
        x = (window.winfo_screenwidth() // 2) - (width // 2)
        y = (window.winfo_screenheight() // 2) - (height // 2)
        window.geometry(f'+{x}+{y}')
    
    def parser(self, pathway):
        data = []
        with open(pathway, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            super_info = soup.find_all(class_='resume-card-content--pA9euQ2yPckXrBh1')
            if super_info:
                for load_vac in super_info:

                    start_time = datetime.datetime.now()
                    file_name = os.path.basename(pathway)
                    file_time_create = datetime.datetime.fromtimestamp(os.path.getctime(pathway))
                    type_file = 'Список резюме'

                    main_info = load_vac.find(class_='column-content--q3SfppwQANVUv38P')
                    name_resume = main_info.find('span', {'data-qa': 'serp-item__title-text'})
                    
                    if name_resume:
                        name_resume = name_resume.text
                    else:
                        name_resume = None
                    # Возраст
                    age = load_vac.find('span', {'data-qa': 'resume-serp__resume-age'})
                    if age:
                        age = age.text
                    else:
                        age = None

                    url_find = main_info.find('a')['href']
                    img_url = load_vac.find_all('img')
                    if img_url:
                        img_text = img_url[0]['src']
                        img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
                    else:
                        img_text = None  
        
                    info_about_time_to_be = main_info.find_all(class_='magritte-text___pbpft_4-1-1 magritte-text_style-secondary___1IU11_4-1-1 magritte-text_typography-label-3-regular___Nhtlp_4-1-1')
                
                    all_activity_dates = []
                    for element in info_about_time_to_be:
                        # Ищем все строки с "Был" или "Была"
                        activity_strings = element.find_all(string=lambda t: isinstance(t, str) and ('Был' in t or 'Была' in t))
                
                        for activity in activity_strings:
                            # Для каждой найденной строки находим следующую дату
                            date_span = activity.find_next('span')
                            if date_span:
                                full_text = f"{activity.strip()} {date_span.text.strip()}"
                                all_activity_dates.append(full_text)
                
                    # Если нашли активности, объединяем их через запятую
                    full_text = ', '.join(all_activity_dates) if all_activity_dates else None
                    full_text = parse_date_string(full_text)
                    short_url = re.match(r'^[^?]+', url_find).group()
        
                    data.append((name_resume, age, url_find, img_text, short_url, full_text))
        
                parsing_end = datetime.datetime.now()
                metadata = (file_name, file_time_create, start_time, parsing_end, type_file, self.current_comment)
                return data, metadata
            
            else:
                # Обработка отдельных резюме
                start_time = datetime.datetime.now()
                file_name = os.path.basename(pathway)
                file_time_create = datetime.datetime.fromtimestamp(os.path.getctime(pathway))
                type_file = 'Отдельное резюме'

                # HTML основная информация
                head = soup.find(class_ = 'header--FIqvP_vS2Y1E2k5a')

                name_resume = head.find('h2', {'data-qa': 'title'})
                if name_resume:
                    name_resume = name_resume.text
                else:
                    name_resume = None
                age = soup.find('span', {'data-qa': 'resume-personal-age'})
                if age:
                    age = age.text
                else:
                    age = None
                # URL
                comments = soup.find_all(string=lambda text: isinstance(text, bs4.Comment))
                url_find = None
                if comments:
                    comment_text = comments[0]
                    match = re.search(r'url=\((\d+)\)(.*)', comment_text)
                    if match:
                        url_find = match.group(2)
                        url_find = url_find.strip()
                
                img_url = soup.find_all('img', attrs={'alt': 'resume photo'})
                if img_url:
                    img_text = img_url[0]['src']
                    img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
                else:
                    img_text = None 
                
                short_url = re.match(r'^[^?]+', url_find).group()
                
                full_text = None
                data.append((name_resume, age, url_find, img_text, short_url, full_text))

                parsing_end = datetime.datetime.now()
                metadata = (file_name, file_time_create, start_time, parsing_end, type_file, self.current_comment)
                return data, metadata

    def load_db(self, data, metadata, session_id):
        try: 
            with psycopg2.connect(**self.DB_CONFIG) as connect:
                with connect.cursor() as cur:
                    
                    cur.execute('''CREATE TABLE IF NOT EXISTS info_res (
                            id SERIAL PRIMARY KEY,
                            session_id INTEGER,
                            name_resume TEXT,
                            age TEXT,
                            full_url TEXT,
                            img_text TEXT,
                            short_url TEXT,
                            visit_time DATE,
                            comment TEXT)''')

                    cur.execute('''CREATE TABLE IF NOT EXISTS parsing_metadata (
                                    id SERIAL PRIMARY KEY,
                                    session_id INTEGER,
                                    html_file_name TEXT,
                                    file_creation_time TIMESTAMP,
                                    parsing_start_time TIMESTAMP,
                                    parsing_end_time TIMESTAMP,
                                    type_file TEXT,
                                    parsing_comment TEXT);''')

                    # Подготавливаем данные для вставки
                    data_with_session = [(session_id, name_resume, age, url, img_text, short_url, visit_time, '') 
                                       for name_resume, age, url, img_text, short_url, visit_time in data]

                    # Добавляем комментарий к метаданным
                    metadata_with_session = (session_id,) + metadata[1:]  # Пропускаем первый элемент (file_name)

                    # Вставляем данные
                    cur.executemany('''INSERT INTO info_res 
                                    (session_id, name_resume, age, full_url, img_text, short_url, visit_time, comment) 
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''', 
                                 data_with_session)

                    cur.execute('''INSERT INTO parsing_metadata 
                                (session_id, html_file_name, file_creation_time, 
                                 parsing_start_time, parsing_end_time, type_file, parsing_comment) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s)''', 
                             metadata_with_session)

                    connect.commit()
                return True

        except Exception as e:
            print(f'Ошибка {e}')
            return False

if __name__ == "__main__":
    app = Application()
    app.mainloop()

Ошибка tuple index out of range
Ошибка tuple index out of range
Ошибка tuple index out of range


In [76]:
import re 
from bs4 import BeautifulSoup
import bs4

data = []
with open('Прораб.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')
    head = soup.find(class_ = 'header--FIqvP_vS2Y1E2k5a')
    resume_main = soup.find('div', class_='resume-wrapper')
    print(resume_main)
    name_resume = head.find('h2', {'data-qa': 'title'})
    if name_resume:
        name_resume = name_resume.text
    else:
        name_resume = None
    comments = soup.find_all(string=lambda text: isinstance(text, bs4.Comment))

    url = None
    if comments:
        comment_text = comments[0]
        match = re.search(r'url=\((\d+)\)(.*)', comment_text)
        if match:
            url = match.group(2)
            url = url.strip()
    print(url)

    age = soup.find('span', {'data-qa': 'resume-personal-age'}).text
    print(age)

    img_url = soup.find_all('img', attrs={'alt': 'resume photo'})
    if img_url:
        img_text = img_url[0]['src']
        img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
    else:
        img_text = None
    print(img_text)
    short_url = re.match(r'^[^?]+', url).group()
    print(short_url)


None
https://hh.ru/resume/07f1b5b6000e38ca92002daafe53624a704878?query=%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80+%D0%BF%D0%B3%D1%81&searchRid=1755156181349e24c68501db7b67b005&hhtmFrom=resume_search_result
20 лет
780517656.jpeg
https://hh.ru/resume/07f1b5b6000e38ca92002daafe53624a704878


In [42]:
from bs4 import BeautifulSoup
import bs4

with open('ТЕСТ.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')
    a = soup.find_all(class_='resume-card-content--pA9euQ2yPckXrBh1')
    if a:
        print('это список резюмешек')
    else:
        print('Это резюмешка')
    

это список резюмешек


In [29]:
match = re.search(r"был (\d{1,2})\xa0(.*)", 'Был сегодня' )
if match:
    day = int(match.group(1))
    month_name = match.group(2).strip
    month_dict = {
        "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
        "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12,}

    if month_name in month_dict:
        month = month_dict[month_name]
        year = datetime.date.today().year
        print(datetime.date(year, month, day))
        

In [30]:
match.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [47]:
import re 

data = []
with open('Тест.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')
for load_vac in soup.find_all(class_='resume-card-content--pA9euQ2yPckXrBh1'):
    start_time = datetime.datetime.now()
    file_name = 'Тест.html'
    img_url = load_vac.find_all('img')
    if img_url:
        img_text = img_url[0]['src']
        img_text = re.findall(r'\d+(?:.jpeg|.png)', img_text)[0]
    else:
        img_text = None                 
    main_info = load_vac.find(class_='column-content--q3SfppwQANVUv38P')
    age = load_vac.find('span', {'data-qa': 'resume-serp__resume-age'})
    print(age.text)

    name_resume = main_info.find('span', {'data-qa': 'serp-item__title-text'}).text
    print(name_resume)
    url_find = main_info.find('a')['href']
    info_about_time_to_be = main_info.find_all(class_='magritte-text___pbpft_4-1-1 magritte-text_style-secondary___1IU11_4-1-1 magritte-text_typography-label-3-regular___Nhtlp_4-1-1')

    all_activity_dates = []
    for element in info_about_time_to_be:
        # Ищем все строки с "Был" или "Была"
        activity_strings = element.find_all(string=lambda t: isinstance(t, str) and ('Был' in t or 'Была' in t))

        for activity in activity_strings:
            # Для каждой найденной строки находим следующую дату
            date_span = activity.find_next('span')
            if date_span:
                full_text = f"{activity.strip()} {date_span.text.strip()}"
                all_activity_dates.append(full_text)

    # Если нашли активности, объединяем их через запятую
    full_text = ', '.join(all_activity_dates) if all_activity_dates else None
    data.append((url_find, full_text, img_url))

    parsing_end = datetime.datetime.now()
    metadata = (file_name, start_time, parsing_end)

20 лет
Начинающий специалист
20 лет
Инженер ПТО
20 лет
Инженер ПГС, инженер-проектировщик
20 лет
Прораб
19 лет
Разнорабочий
20 лет
Начинающий специалист
20 лет
Прораб
20 лет
Инженер-проектировщик ПГС
20 лет
Архитектор-дизайнер
20 лет
Инженер
19 лет
Монтажник
20 лет
Инженер проектировщик
20 лет
Мастер СМР
20 лет
Помощник инженера ПТО , инженер ПТО , помощник прораба
20 лет
Прораб
19 лет
Архитектор
20 лет
Инженер ПТО
20 лет
Инженер ПТО
19 лет
Инженер-строитель
20 лет
Строитель 
19 лет
Строитель
19 лет
Прораб, помощник прораба, мастер СМР, Инженер-проектировщик, сметчик, помощник ГИПа
20 лет
Инженер-проектировщик
19 лет
Инженер-строитель ПГС( студент)
20 лет
Инженер
20 лет
Инженер-строитель
19 лет
Дизайнер интерьеров
20 лет
Инженер-строитель ПГС
20 лет
Инженер ПТО


In [69]:
main_info
name_resume = main_info.find('span', {'data-qa': 'serp-item__title-text'}).text
print(name_resume)

Инженер ПТО


In [37]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import psycopg2
import webbrowser
from typing import List, Callable

class DataChangeManager:
    """Класс для управления уведомлениями об изменениях данных"""
    def __init__(self):
        self._subscribers: List[Callable] = []
    
    def subscribe(self, callback: Callable):
        """Подписать функцию на уведомления об изменениях"""
        self._subscribers.append(callback)
    
    def unsubscribe(self, callback: Callable):
        """Отписать функцию от уведомлений"""
        if callback in self._subscribers:
            self._subscribers.remove(callback)
    
    def notify(self):
        """Уведомить всех подписчиков об изменении данных"""
        for callback in self._subscribers:
            callback()

# Создаем глобальный менеджер изменений
data_change_manager = DataChangeManager()

DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'hh_test',
    'user': 'postgres',
    'password': '123'
}

# Глобальные переменные
df = pd.DataFrame()
comments = {}

def load_data():
    """Загружает данные из PostgreSQL в DataFrame"""
    global df, comments
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            # Проверяем существование колонки comment
            with connect.cursor() as cursor:
                cursor.execute("""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name='info_res' AND column_name='comment'
                """)
                if not cursor.fetchone():
                    cursor.execute("ALTER TABLE info_res ADD COLUMN comment TEXT")
                    connect.commit()
            
            df = pd.read_sql('''select info_res.session_id, name_resume, full_url, img_text, short_url, visit_time, file_creation_time, comment from info_res
                             Left join parsing_metadata on info_res.session_id = parsing_metadata.session_id
                             ORDER BY info_res.id ASC''', connect)
            
            comments = {f"I{idx}": row['comment'] for idx, row in df.iterrows() if pd.notna(row['comment'])}
            
        return df
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
        return pd.DataFrame()

def save_comment_to_db(row_id, comment):
    """Сохраняет комментарий в базе данных (пустая строка при удалении)"""
    try:
        with psycopg2.connect(**DB_CONFIG) as connect:
            with connect.cursor() as cursor:
                # Используем прямое обновление по индексу строки (не по ID)
                cursor.execute(
                    "UPDATE info_res SET comment = %s WHERE ctid = (SELECT ctid FROM info_res ORDER BY id LIMIT 1 OFFSET %s)",
                    (comment, row_id))
                connect.commit()
        return True
    except Exception as e:
        messagebox.showerror("Ошибка", f"Не удалось сохранить комментарий: {e}")
        return False

class CommentEditor:
    """Окно редактирования комментария"""
    def __init__(self, parent, item, current_value):
        self.editor = tk.Toplevel(parent)
        self.editor.title("Редактирование комментария")
        self.editor.geometry("400x300")
        self.editor.grab_set()
        
        self.parent = parent
        self.item = item
        self.row_id = int(item[1:])  # Получаем индекс строки
        
        main_frame = ttk.Frame(self.editor)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        text_scroll = ttk.Scrollbar(text_frame)
        text_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.text_area = tk.Text(text_frame, wrap=tk.WORD, yscrollcommand=text_scroll.set, height=10)
        self.text_area.insert("1.0", current_value)
        self.text_area.pack(fill=tk.BOTH, expand=True)
        
        text_scroll.config(command=self.text_area.yview)
        
        button_frame = ttk.Frame(main_frame, height=30)
        button_frame.pack(fill=tk.X, pady=(10, 0))
        button_frame.pack_propagate(False)
        
        # Кнопки
        save_btn = ttk.Button(button_frame, text="Сохранить", width=10,
                            command=self.save_changes)
        clear_btn = ttk.Button(button_frame, text="Очистить", width=10,
                             command=self.clear_comment)
        cancel_btn = ttk.Button(button_frame, text="Отмена", width=10,
                              command=self.editor.destroy)
        
        save_btn.pack(side=tk.LEFT, padx=5)
        clear_btn.pack(side=tk.LEFT, padx=5)
        cancel_btn.pack(side=tk.RIGHT, padx=5)
        
        self.text_area.focus_set()
    
    def save_changes(self):
        """Сохраняет изменения комментария"""
        new_comment = self.text_area.get("1.0", "end-1c")
        
        if save_comment_to_db(self.row_id, new_comment):
            # Обновляем DataFrame и словарь комментариев
            df.at[self.row_id, 'comment'] = new_comment
            if new_comment:
                comments[self.item] = new_comment
            elif self.item in comments:
                del comments[self.item]
            
            # Уведомляем всех подписчиков об изменении
            data_change_manager.notify()
            self.editor.destroy()
    
    def clear_comment(self):
        """Очищает комментарий (устанавливает пустую строку)"""
        if save_comment_to_db(self.row_id, ""):
            # Обновляем DataFrame и словарь комментариев
            df.at[self.row_id, 'comment'] = ""
            if self.item in comments:
                del comments[self.item]
            
            # Уведомляем всех подписчиков об изменении
            data_change_manager.notify()
            self.editor.destroy()

class ApplicationWindow:
    """Класс основного окна приложения"""
    def __init__(self, root):
        self.root = root
        self.root.title("Фильтр резюме HH с комментариями")
        self.root.geometry("1000x700")
        
        # Сначала загружаем данные
        self.load_initial_data()
        
        # Затем настраиваем UI с уже загруженными данными
        self.setup_ui()
        
        # Подписываемся на уведомления об изменениях
        data_change_manager.subscribe(self.on_data_changed)
        
        # При закрытии окна отписываемся
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
    
    def on_close(self):
        """Обработчик закрытия окна"""
        data_change_manager.unsubscribe(self.on_data_changed)
        self.root.destroy()
    
    def on_data_changed(self):
        """Обработчик изменения данных - перезагружает данные и обновляет таблицу"""
        self.load_initial_data()
        self.update_table(df)
    
    def setup_ui(self):
        """Настройка пользовательского интерфейса"""
        # Панель фильтров (остается без изменений)
        self.filter_frame = ttk.Frame(self.root, padding="10")
        self.filter_frame.pack(fill=tk.X)

        self.group_by_short_url_var = tk.BooleanVar()
        self.group_by_short_url_check = ttk.Checkbutton(
            self.filter_frame, 
            text="Группировать по короткому url", 
            variable=self.group_by_short_url_var,
            command=lambda: self.update_table(df)
        )
        self.group_by_short_url_check.grid(row=4, column=2, padx=5, pady=5)

        ttk.Label(self.filter_frame, text="Ключевое слово в URL:").grid(row=0, column=0, padx=5, pady=5)
        self.keyword_entry = ttk.Entry(self.filter_frame)
        self.keyword_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата от:").grid(row=1, column=0, padx=5, pady=5)
        self.min_date_entry = ttk.Entry(self.filter_frame)
        self.min_date_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Дата до:").grid(row=2, column=0, padx=5, pady=5)
        self.max_date_entry = ttk.Entry(self.filter_frame)
        self.max_date_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        ttk.Label(self.filter_frame, text="Фильтр по комментариям:").grid(row=3, column=0, padx=5, pady=5)
        self.comment_filter_entry = ttk.Entry(self.filter_frame)
        self.comment_filter_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")

        self.filter_button = ttk.Button(self.filter_frame, text="Применить фильтры", command=self.filter_data)
        self.filter_button.grid(row=4, column=0, columnspan=2, pady=10)

        # Контейнер для таблицы и прокруток
        self.table_container = ttk.Frame(self.root)
        self.table_container.pack(fill=tk.BOTH, expand=True)

        # Горизонтальная прокрутка
        self.xscrollbar = ttk.Scrollbar(self.table_container, orient="horizontal")
        self.xscrollbar.pack(side=tk.BOTTOM, fill=tk.X)

        # Вертикальная прокрутка
        self.yscrollbar = ttk.Scrollbar(self.table_container, orient="vertical")
        self.yscrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Таблица
        columns = list(df.columns)
        self.table = ttk.Treeview(
            self.table_container,
            columns=columns,
            show="headings",
            yscrollcommand=self.yscrollbar.set,
            xscrollcommand=self.xscrollbar.set
        )

        # Настраиваем колонки
        for col in columns:
            self.table.heading(col, text=col)
            self.table.column(col, width=120, stretch=False)  # stretch=False чтобы колонки не растягивались

        style = ttk.Style()
        style.configure('Separator.Treeview', background='gray', rowheight=1)
        self.table.tag_configure('separator', background='gray', font=('Helvetica', 1))

        # Упаковываем таблицу
        self.table.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Настраиваем прокрутки
        self.yscrollbar.config(command=self.table.yview)
        self.xscrollbar.config(command=self.table.xview)

        # Привязка событий
        self.table.bind('<Double-1>', self.on_double_click)
        self.table.bind('<Button-1>', self.on_link_click)

        # Заполняем таблицу данными
        self.update_table(df)
        
    def load_initial_data(self):
        """Загрузка начальных данных"""
        global df, comments
        df = load_data()
        if 'comment' not in df.columns:
            df['comment'] = ''
    
    def on_double_click(self, event):
        """Обработчик двойного клика для редактирования комментариев"""
        item = self.table.identify_row(event.y)
        column = self.table.identify_column(event.x)
        
        # Проверяем, что клик был по строке и по столбцу комментариев
        if item and column == '#{}'.format(df.columns.get_loc('comment') + 1):
            current_value = comments.get(item, "")
            CommentEditor(self.root, item, current_value)
    
    def on_link_click(self, event):
        """Открывает ссылку при клике на full_url"""
        item = self.table.identify_row(event.y)
        col = self.table.identify_column(event.x)
        
        if self.table.heading(col)['text'] == 'full_url':
            url = self.table.item(item, 'values')[df.columns.get_loc('full_url')]
            if url and isinstance(url, str) and url.startswith(('http://', 'https://')):
                webbrowser.open(url)
    
    def filter_data(self):
        """Фильтрует данные с учетом комментариев"""
        keyword = self.keyword_entry.get().strip().lower()
        min_date = self.min_date_entry.get()
        max_date = self.max_date_entry.get()
        comment_filter = self.comment_filter_entry.get().strip().lower()

        filtered_df = df.copy()
        
        # Фильтр по URL
        if keyword:
            filtered_df = filtered_df[filtered_df['full_url'].str.lower().str.contains(keyword, na=False)]
        
        # Фильтр по дате
        if min_date:
            filtered_df = filtered_df[filtered_df['visit_time'] >= min_date]
        if max_date:
            filtered_df = filtered_df[filtered_df['visit_time'] <= max_date]
        
        # Фильтр по комментариям
        if comment_filter:
            db_filtered = filtered_df['comment'].str.lower().str.contains(comment_filter, na=False)
            comment_filtered_items = [
                int(i.lstrip('I')) for i, comment in comments.items() 
                if comment_filter in comment.lower()
            ]
            combined_filter = db_filtered | filtered_df.index.isin(comment_filtered_items)
            filtered_df = filtered_df[combined_filter]

        self.update_table(filtered_df)
    
    def update_table(self, dataframe):
        """Обновляет таблицу с учетом комментариев и группировки по short_url"""
        # Сохраняем текущее состояние фильтров
        current_filters = {
            'keyword': self.keyword_entry.get().strip().lower(),
            'min_date': self.min_date_entry.get(),
            'max_date': self.max_date_entry.get(),
            'comment_filter': self.comment_filter_entry.get().strip().lower()
        }
        
        # Применяем фильтры к обновленным данным
        filtered_df = dataframe.copy()
        
        # Фильтр по URL
        if current_filters['keyword']:
            filtered_df = filtered_df[filtered_df['full_url'].str.lower().str.contains(
                current_filters['keyword'], na=False)]
        
        # Фильтр по дате
        if current_filters['min_date']:
            filtered_df = filtered_df[filtered_df['visit_time'] >= current_filters['min_date']]
        if current_filters['max_date']:
            filtered_df = filtered_df[filtered_df['visit_time'] <= current_filters['max_date']]
        
        # Фильтр по комментариям
        if current_filters['comment_filter']:
            filtered_df = filtered_df[
                filtered_df['comment'].str.lower().str.contains(
                    current_filters['comment_filter'], na=False)
            ]
        
        # Очищаем таблицу
        self.table.delete(*self.table.get_children())
        
        # Если checkbox включен, группируем по short_url
        if self.group_by_short_url_var.get():
            # Сортируем по short_url для группировки
            filtered_df = filtered_df.sort_values('short_url')
            
            prev_short_url = None
            
            for index, row in filtered_df.iterrows():
                item_id = f"I{index}"
                values = list(row)
                current_short_url = row['short_url']
                
                # Если short_url изменился и это не первая строка, добавляем разделитель
                if prev_short_url is not None and current_short_url != prev_short_url:
                    # Добавляем разделитель (пустую строку с особым тегом)
                    separator_id = f"sep_{prev_short_url}_{current_short_url}"
                    self.table.insert("", "end", iid=separator_id, values=[""]*len(values), tags=('separator',))
                
                # Вставляем строку данных
                self.table.insert("", "end", iid=item_id, values=values)
                
                prev_short_url = current_short_url
        else:
            # Обычное отображение без группировки
            for index, row in filtered_df.iterrows():
                item_id = f"I{index}"
                values = list(row)
                self.table.insert("", "end", iid=item_id, values=values)

def create_new_window():
    """Создает новое окно приложения"""
    new_root = tk.Toplevel()
    new_root.title("Фильтр резюме HH с комментариями")
    new_root.geometry("1000x700")
    ApplicationWindow(new_root)

def main():
    """Основная функция запуска приложения"""
    root = tk.Tk()
    app = ApplicationWindow(root)
    
    # Добавляем кнопку для создания нового окна
    new_window_btn = ttk.Button(root, text="Новое окно", command=create_new_window)
    new_window_btn.pack(side=tk.BOTTOM, pady=10)
    
    root.mainloop()

if __name__ == "__main__":
    main()

C:\Users\sekin\AppData\Local\Temp\ipykernel_22836\1624253886.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''select info_res.session_id, name_resume, full_url, img_text, short_url, visit_time, file_creation_time, comment from info_res


In [45]:
import tkinter as tk
from tkinter import ttk, messagebox
import psycopg2
from PIL import Image, ImageTk
import os
import webbrowser
import pandas as pd

class PhotoGalleryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Галерея резюме")
        self.root.geometry("1200x800")
        self.root.minsize(800, 600)
        
        # Конфигурация базы данных
        self.db_config = {
            'host': 'localhost',
            'port': '5432',
            'database': 'postgres',
            'user': 'postgres',
            'password': '123'
        }
        
        # Путь к папке с фотографиями
        self.photos_base_path = r"C:\Users\SportGroup1\Desktop"
        
        # Текущая страница
        self.current_page = 0
        self.items_per_page = 9
        self.filter_mode = False
        self.current_filter = ""
        self.image_cache = {}
        self.current_display_data = pd.DataFrame()
        self.session_min = 0
        self.session_max = 0
        
        # Загружаем данные
        self.load_data()
        
        # Создаем интерфейс
        self.create_widgets()
        
        # Показываем первую страницу
        self.show_page()
    
    def load_data(self):
        """Загружает данные из базы данных"""
        try:
            with psycopg2.connect(**self.db_config) as conn:
                # Получаем данные о резюме с информацией о сессии
                self.resumes = pd.read_sql("""
                    SELECT i.*, p.html_file_name, p.parsing_comment as session_comment
                    FROM info_res i
                    LEFT JOIN parsing_metadata p ON i.session_id = p.session_id
                    WHERE i.img_text IS NOT NULL AND i.img_text != ''
                    ORDER BY i.id
                """, conn)
                
                # Получаем минимальный и максимальный номер сессии
                session_range = pd.read_sql("""
                    SELECT MIN(session_id) as min_session, MAX(session_id) as max_session 
                    FROM info_res
                """, conn)
                
                self.session_min = session_range.iloc[0]['min_session'] or 0
                self.session_max = session_range.iloc[0]['max_session'] or 0
                
                # Фильтруем записи без фотографий
                self.resumes = self.resumes[self.resumes['img_text'].notna() & 
                                          (self.resumes['img_text'] != '')]
                
                # Инициализируем текущие отображаемые данные
                self.current_display_data = self.resumes.copy()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось загрузить данные: {e}")
            self.resumes = pd.DataFrame()
            self.current_display_data = pd.DataFrame()
    
    def create_widgets(self):
        """Создает элементы интерфейса"""
        # Основной контейнер для всего содержимого
        self.main_container = ttk.Frame(self.root)
        self.main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Фрейм для фильтров
        self.filter_frame = ttk.Frame(self.main_container)
        self.filter_frame.pack(fill=tk.X, pady=5)
        
        # Поле для фильтрации по комментариям
        ttk.Label(self.filter_frame, text="Фильтр по комментариям:").pack(side=tk.LEFT, padx=5)
        self.filter_entry = ttk.Entry(self.filter_frame, width=20)
        self.filter_entry.pack(side=tk.LEFT, padx=5)
        
        # Фильтр по номеру сессии
        ttk.Label(self.filter_frame, text="С сессии:").pack(side=tk.LEFT, padx=5)
        self.session_from = ttk.Entry(self.filter_frame, width=5)
        self.session_from.pack(side=tk.LEFT, padx=5)
        self.session_from.insert(0, str(self.session_min))
        
        ttk.Label(self.filter_frame, text="по:").pack(side=tk.LEFT, padx=0)
        self.session_to = ttk.Entry(self.filter_frame, width=5)
        self.session_to.pack(side=tk.LEFT, padx=5)
        self.session_to.insert(0, str(self.session_max))
        
        # Кнопка применения фильтра
        self.filter_btn = ttk.Button(self.filter_frame, text="Применить фильтр", 
                                   command=self.apply_filter)
        self.filter_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка сброса фильтра
        self.reset_filter_btn = ttk.Button(self.filter_frame, text="Сбросить фильтр",
                                         command=self.reset_filter)
        self.reset_filter_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка массового добавления комментария
        self.bulk_comment_btn = ttk.Button(self.filter_frame, text="Добавить комментарий ко всем",
                                         command=self.bulk_add_comment)
        self.bulk_comment_btn.pack(side=tk.RIGHT, padx=5)
        
        # Фрейм для фотографий с прокруткой
        self.photo_container = ttk.Frame(self.main_container)
        self.photo_container.pack(fill=tk.BOTH, expand=True)
        
        # Canvas и Scrollbar
        self.canvas = tk.Canvas(self.photo_container)
        self.scrollbar = ttk.Scrollbar(self.photo_container, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )
        
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        
        # Размещаем canvas и scrollbar
        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")
        
        # Создаем сетку 3x3 для фотографий
        self.photo_frames = []
        self.photo_labels = []
        self.name_labels = []
        
        for i in range(3):
            self.scrollable_frame.grid_rowconfigure(i, weight=1)
            for j in range(3):
                self.scrollable_frame.grid_columnconfigure(j, weight=1)
                
                # Фрейм для фотографии и подписи
                frame = ttk.Frame(self.scrollable_frame, relief=tk.RAISED, borderwidth=1,
                                 width=300, height=350)
                frame.grid_propagate(False)
                frame.grid(row=i, column=j, padx=10, pady=10, sticky="nsew")
                
                # Метка для фотографии
                photo_label = tk.Label(frame)
                photo_label.pack(fill=tk.BOTH, expand=True)
                
                # Метка для названия резюме
                name_label = ttk.Label(frame, text="", wraplength=200, 
                                    anchor=tk.CENTER, justify=tk.CENTER)
                name_label.pack(fill=tk.X, pady=(5, 0))
                
                self.photo_frames.append(frame)
                self.photo_labels.append(photo_label)
                self.name_labels.append(name_label)
        
        # Фрейм для пагинации (фиксированный внизу)
        self.pagination_frame = ttk.Frame(self.main_container)
        self.pagination_frame.pack(fill=tk.X, pady=(10, 0))
        
        # Кнопки пагинации
        self.prev_btn = ttk.Button(self.pagination_frame, text="◄ Назад", width=15, 
                                 command=self.prev_page)
        self.prev_btn.pack(side=tk.LEFT, padx=20)
        
        self.page_label = ttk.Label(self.pagination_frame, text="Страница 1 из 1", 
                                  font=('Arial', 10))
        self.page_label.pack(side=tk.LEFT, expand=True)
        
        self.next_btn = ttk.Button(self.pagination_frame, text="Вперед ►", width=15,
                                 command=self.next_page)
        self.next_btn.pack(side=tk.RIGHT, padx=20)
    
    def apply_filter(self):
        """Применяет фильтр по комментариям и номеру сессии"""
        filter_text = self.filter_entry.get().strip()
        session_from = self.session_from.get().strip()
        session_to = self.session_to.get().strip()
        
        try:
            session_from = int(session_from) if session_from else self.session_min
            session_to = int(session_to) if session_to else self.session_max
        except ValueError:
            messagebox.showwarning("Ошибка", "Номера сессий должны быть целыми числами")
            return
        
        self.current_filter = filter_text.lower()
        self.filter_mode = True
        self.current_page = 0
        
        # Фильтруем данные
        filtered_resumes = []
        for _, row in self.resumes.iterrows():
            match = True
            
            # Фильтр по номеру сессии
            if session_from is not None and session_to is not None:
                if not (session_from <= row['session_id'] <= session_to):
                    match = False
            
            # Фильтр по комментарию
            if match and self.current_filter:
                if pd.isna(row['comment']) or self.current_filter not in str(row['comment']).lower():
                    match = False
            
            if match:
                filtered_resumes.append(row)
        
        self.current_display_data = pd.DataFrame(filtered_resumes)
        self.image_cache.clear()
        self.show_page()
    
    def reset_filter(self):
        """Сбрасывает фильтр и показывает все резюме"""
        self.filter_mode = False
        self.current_filter = ""
        self.filter_entry.delete(0, tk.END)
        self.session_from.delete(0, tk.END)
        self.session_from.insert(0, str(self.session_min))
        self.session_to.delete(0, tk.END)
        self.session_to.insert(0, str(self.session_max))
        self.current_page = 0
        self.current_display_data = self.resumes.copy()
        self.image_cache.clear()
        self.show_page()
    
    def bulk_add_comment(self):
        """Добавляет комментарий ко всем отфильтрованным резюме"""
        if not self.filter_mode:
            messagebox.showwarning("Предупреждение", "Сначала примените фильтр")
            return
        
        # Создаем окно для ввода комментария
        comment_window = tk.Toplevel(self.root)
        comment_window.title("Добавить комментарий ко всем")
        comment_window.geometry("400x300")
        
        ttk.Label(comment_window, text="Введите комментарий:").pack(pady=10)
        
        comment_text = tk.Text(comment_window, height=10, width=40, wrap=tk.WORD)
        comment_text.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        def save_bulk_comment():
            comment = comment_text.get("1.0", tk.END).strip()
            if not comment:
                messagebox.showwarning("Предупреждение", "Введите текст комментария")
                return
            
            try:
                with psycopg2.connect(**self.db_config) as conn:
                    with conn.cursor() as cursor:
                        # Получаем все ID из текущих отображаемых данных
                        filtered_ids = list(self.current_display_data['id'])
                        
                        if not filtered_ids:
                            messagebox.showinfo("Информация", "Нет резюме для обновления")
                            return
                        
                        # Обновляем комментарии
                        cursor.executemany("""
                            UPDATE info_res 
                            SET comment = %s 
                            WHERE id = %s
                        """, [(comment, id) for id in filtered_ids])
                        
                        conn.commit()
                
                messagebox.showinfo("Успех", f"Комментарий добавлен к {len(filtered_ids)} резюме")
                comment_window.destroy()
                self.load_data()  # Полная перезагрузка данных
                self.show_page()
            
            except Exception as e:
                messagebox.showerror("Ошибка", f"Не удалось обновить комментарии: {e}")
        
        ttk.Button(comment_window, text="Сохранить", command=save_bulk_comment).pack(pady=10)
    
    def show_page(self):
        """Отображает текущую страницу с фотографиями"""
        # Очищаем предыдущие фотографии
        for frame in self.photo_frames:
            frame.grid_remove()
            for widget in frame.winfo_children():
                if isinstance(widget, tk.Label):
                    widget.config(image='', text='')
                    widget.unbind("<Button-1>")
        
        # Вычисляем диапазон элементов для текущей страницы
        total_items = len(self.current_display_data)
        start_idx = self.current_page * self.items_per_page
        end_idx = min(start_idx + self.items_per_page, total_items)
        
        # Обновляем метку страницы
        total_pages = max(1, (total_items + self.items_per_page - 1) // self.items_per_page)
        self.page_label.config(text=f"Страница {self.current_page + 1} из {total_pages}")
        
        # Обновляем состояние кнопок пагинации
        self.prev_btn['state'] = 'normal' if self.current_page > 0 else 'disabled'
        self.next_btn['state'] = 'normal' if end_idx < total_items else 'disabled'
        
        # Отображаем фотографии для текущей страницы
        displayed_items = 0
        for idx in range(start_idx, end_idx):
            resume = self.current_display_data.iloc[idx]
            photo_path = self.get_photo_path(resume['html_file_name'], resume['img_text'])
            
            # Пропускаем записи без фотографий
            if not photo_path or not os.path.exists(photo_path):
                continue
                
            frame = self.photo_frames[displayed_items]
            photo_label = self.photo_labels[displayed_items]
            name_label = self.name_labels[displayed_items]
            frame.grid()  # Показываем фрейм
            
            try:
                # Загружаем изображение с кэшированием
                cache_key = f"{resume['html_file_name']}_{resume['img_text']}"
                
                if cache_key not in self.image_cache:
                    img = Image.open(photo_path)
                    
                    # Фиксированные размеры для предсказуемости
                    target_width, target_height = 280, 280
                    
                    # Масштабирование с сохранением пропорций
                    img_ratio = img.width / img.height
                    if img_ratio > 1:
                        new_width = target_width
                        new_height = int(target_width / img_ratio)
                    else:
                        new_height = target_height
                        new_width = int(target_height * img_ratio)
                    
                    img = img.resize((new_width, new_height), Image.LANCZOS)
                    self.image_cache[cache_key] = ImageTk.PhotoImage(img)
                
                photo_img = self.image_cache[cache_key]
                
                # Обновляем виджет изображения
                photo_label.config(image=photo_img)
                photo_label.image = photo_img
                
                # Добавляем название резюме
                name = resume['name_resume']
                name_display = (name[:47] + "...") if len(name) > 50 else name
                
                # Добавляем иконку комментария, если он есть
                comment_indicator = " 💬" if pd.notna(resume['comment']) and str(resume['comment']).strip() else ""
                session_info = f" (Сессия: {resume['session_id']})"
                name_label.config(text=name_display + session_info + comment_indicator)
                
                # Привязываем обработчики клика
                photo_label.bind("<Button-1>", lambda e, r=resume.to_dict(): self.show_resume_details(r))
                name_label.bind("<Button-1>", lambda e, r=resume.to_dict(): self.show_resume_details(r))
                frame.bind("<Button-1>", lambda e, r=resume.to_dict(): self.show_resume_details(r))
                
                displayed_items += 1
                if displayed_items >= self.items_per_page:
                    break
                    
            except Exception as e:
                print(f"Ошибка загрузки фото {photo_path}: {e}")
                continue
        
        # Обновляем прокрутку
        self.canvas.yview_moveto(0)
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
    
    def get_photo_path(self, html_file_name, img_text):
        """Возвращает точный путь к фотографии кандидата"""
        if not html_file_name or pd.isna(html_file_name) or not img_text or pd.isna(img_text):
            return None
        
        try:
            # Удаляем расширение .html
            base_name = os.path.splitext(html_file_name)[0]
            photos_dir = os.path.join(self.photos_base_path, f"{base_name}_files")
            
            if not os.path.exists(photos_dir):
                return None
            
            # Получаем чистое имя файла из img_text
            img_filename = os.path.basename(str(img_text).split('?')[0].strip('"\''))
            
            # 1. Попытка найти точное совпадение
            exact_path = os.path.join(photos_dir, img_filename)
            if os.path.exists(exact_path):
                return exact_path
            
            # 2. Если точного совпадения нет, ищем файл с тем же именем, но другим регистром
            for file in os.listdir(photos_dir):
                if file.lower() == img_filename.lower():
                    return os.path.join(photos_dir, file)
            
            return None
            
        except Exception as e:
            print(f"Ошибка поиска фото: {e}")
            return None
    
    def show_resume_details(self, resume):
        """Показывает детальную информацию о резюме"""
        detail_window = tk.Toplevel(self.root)
        detail_window.title(f"Резюме: {resume['name_resume']}")
        detail_window.geometry("700x700")
        
        main_frame = ttk.Frame(detail_window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Фрейм для фотографии
        photo_frame = ttk.Frame(main_frame)
        photo_frame.pack(fill=tk.X, pady=10)
        
        try:
            photo_path = self.get_photo_path(resume['html_file_name'], resume['img_text'])
            if photo_path and os.path.exists(photo_path):
                img = Image.open(photo_path)
                img.thumbnail((200, 200))
                photo_img = ImageTk.PhotoImage(img)
                
                photo_label = tk.Label(photo_frame, image=photo_img)
                photo_label.image = photo_img
                photo_label.pack()
        except Exception as e:
            print(f"Ошибка загрузки фото: {e}")
        
        # Фрейм для информации
        info_frame = ttk.Frame(main_frame)
        info_frame.pack(fill=tk.BOTH, expand=True)
        
        # Создаем Notebook для вкладок
        notebook = ttk.Notebook(info_frame)
        notebook.pack(fill=tk.BOTH, expand=True)
        
        # Вкладка с основной информацией
        general_tab = ttk.Frame(notebook)
        notebook.add(general_tab, text="Основная информация")
        
        # Добавляем информацию о резюме
        fields = [
            ("Название резюме:", resume['name_resume']),
            ("Номер сессии:", resume['session_id']),
            ("Дата посещения:", resume['visit_time']),
            ("URL:", resume['full_url'])
        ]
        
        for i, (label, value) in enumerate(fields):
            ttk.Label(general_tab, text=label, font=('Arial', 10, 'bold')).grid(
                row=i, column=0, sticky=tk.W, padx=5, pady=2)
            
            # Для URL делаем кликабельную ссылку
            if label == "URL:":
                url_label = ttk.Label(general_tab, text=value, foreground="blue", cursor="hand2")
                url_label.grid(row=i, column=1, sticky=tk.W, padx=5, pady=2)
                url_label.bind("<Button-1>", lambda e, url=value: webbrowser.open(url))
            else:
                ttk.Label(general_tab, text=value, wraplength=400).grid(
                    row=i, column=1, sticky=tk.W, padx=5, pady=2)
        
        # Вкладка с комментариями
        comments_tab = ttk.Frame(notebook)
        notebook.add(comments_tab, text="Комментарии")
        
        # Комментарий к резюме
        ttk.Label(comments_tab, text="Комментарий к резюме:", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=5)
        
        self.resume_comment_text = tk.Text(comments_tab, height=5, width=60, wrap=tk.WORD)
        self.resume_comment_text.pack(fill=tk.X, pady=5)
        self.resume_comment_text.insert(tk.END, resume.get('comment', '') if pd.notna(resume.get('comment')) else "")
        
        # Комментарий к сессии
        ttk.Label(comments_tab, text="Комментарий к сессии:", font=('Arial', 10, 'bold')).pack(anchor=tk.W, pady=5)
        
        self.session_comment_text = tk.Text(comments_tab, height=5, width=60, wrap=tk.WORD)
        self.session_comment_text.pack(fill=tk.X, pady=5)
        self.session_comment_text.insert(tk.END, resume.get('session_comment', '') if pd.notna(resume.get('session_comment')) else "")
        
        # Фрейм для кнопок
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill=tk.X, pady=10)
        
        # Кнопка сохранения комментария
        save_btn = ttk.Button(buttons_frame, text="Сохранить комментарии", 
                             command=lambda: self.save_comments(resume['id'], resume['session_id'], detail_window))
        save_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка очистки комментария
        clear_btn = ttk.Button(buttons_frame, text="Удалить комментарии",
                             command=lambda: self.clear_comments(resume['id'], resume['session_id'], detail_window))
        clear_btn.pack(side=tk.LEFT, padx=5)
        
        # Кнопка закрытия
        close_btn = ttk.Button(buttons_frame, text="Закрыть", command=detail_window.destroy)
        close_btn.pack(side=tk.RIGHT, padx=5)
    
    def save_comments(self, resume_id, session_id, detail_window=None):
        """Сохраняет комментарии к резюме и сессии"""
        resume_comment = str(self.resume_comment_text.get("1.0", tk.END)).strip()
        session_comment = str(self.session_comment_text.get("1.0", tk.END)).strip()
        
        try:
            with psycopg2.connect(**self.db_config) as conn:
                with conn.cursor() as cursor:
                    # Обновляем комментарий к резюме
                    if not resume_comment:
                        cursor.execute("""
                            UPDATE info_res 
                            SET comment = NULL 
                            WHERE id = %s
                        """, (resume_id,))
                    else:
                        cursor.execute("""
                            UPDATE info_res 
                            SET comment = %s 
                            WHERE id = %s
                        """, (resume_comment, resume_id))
                    
                    # Обновляем комментарий к сессии
                    if not session_comment:
                        cursor.execute("""
                            UPDATE parsing_metadata 
                            SET parsing_comment = NULL 
                            WHERE session_id = %s
                        """, (session_id,))
                    else:
                        cursor.execute("""
                            UPDATE parsing_metadata 
                            SET parsing_comment = %s 
                            WHERE session_id = %s
                        """, (session_comment, session_id))
                    
                    conn.commit()
            
            messagebox.showinfo("Успех", "Комментарии успешно сохранены")
            self.load_data()  # Полная перезагрузка данных
            if detail_window:
                detail_window.destroy()
                
        except Exception as e:
            messagebox.showerror("Ошибка", f"Не удалось сохранить комментарии: {e}")
    
    def clear_comments(self, resume_id, session_id, detail_window=None):
        """Удаляет комментарии к резюме и сессии"""
        if messagebox.askyesno("Подтверждение", "Вы уверены, что хотите удалить все комментарии?"):
            try:
                with psycopg2.connect(**self.db_config) as conn:
                    with conn.cursor() as cursor:
                        # Удаляем комментарий к резюме
                        cursor.execute("""
                            UPDATE info_res 
                            SET comment = NULL 
                            WHERE id = %s
                        """, (resume_id,))
                        
                        # Удаляем комментарий к сессии
                        cursor.execute("""
                            UPDATE parsing_metadata 
                            SET parsing_comment = NULL 
                            WHERE session_id = %s
                        """, (session_id,))
                        
                        conn.commit()
                
                self.resume_comment_text.delete("1.0", tk.END)
                self.session_comment_text.delete("1.0", tk.END)
                messagebox.showinfo("Успех", "Комментарии удалены")
                self.load_data()  # Полная перезагрузка данных
                if detail_window:
                    detail_window.destroy()
                    
            except Exception as e:
                messagebox.showerror("Ошибка", f"Не удалось удалить комментарии: {e}")
    
    def prev_page(self):
        """Переход на предыдущую страницу"""
        if self.current_page > 0:
            self.current_page -= 1
            self.show_page()
            self.canvas.yview_moveto(0)
    
    def next_page(self):
        """Переход на следующую страницу"""
        if (self.current_page + 1) * self.items_per_page < len(self.current_display_data):
            self.current_page += 1
            self.show_page()
            self.canvas.yview_moveto(0)

if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoGalleryApp(root)
    root.mainloop()

C:\Users\SportGroup1\AppData\Local\Temp\ipykernel_16804\1558927328.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  self.resumes = pd.read_sql("""
C:\Users\SportGroup1\AppData\Local\Temp\ipykernel_16804\1558927328.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  session_range = pd.read_sql("""
